In [1]:
using Plots
using StatsPlots
using CSV
using DataFrames
using Statistics
plotly()
import Base.Threads
Threads.nthreads()
#packages we need
using DifferentialEquations
using Plots 
using CSV
using Distributions
using Random
using DataFrames
using JLD2
using FileIO
using StatsPlots
using Plots.PlotMeasures
using Statistics
plotly();

┌ Warning: For saving to png with the `Plotly` backend `PlotlyBase` and `PlotlyKaleido` need to be installed.
│   err =
│    ArgumentError: Package PlotlyBase not found in current path.
│    - Run `import Pkg; Pkg.add("PlotlyBase")` to install the PlotlyBase package.
└ @ Plots ~/.julia/packages/Plots/M4dfL/src/backends.jl:545


In [17]:
include("plotAllSpecies2.jl")
include("variableNames.jl");

In [3]:
folder="output/IKKNIKbasalscanforheatmap"

"output/IKKNIKbasalscanforheatmap"

In [24]:
function stackedBar(speciesToPlot,conditionsToPlot,colorArray,first_cell,last_cell,folder,hoursToPlot,maxValOfYAxis,qualityScaling)
    for species in speciesToPlot
        #thisPlot=plot(title=species)
        thisPlotStd=plot(title=species*" avg")
        boxPlotAll=plot(title=species*" ss")
        boxPlotAllTC=plot(title=species*"avg TC")
        conditionIndex=1
        meansOfAllConditions=zeros(1,length(conditionsToPlot))
        stdOfAllConditions=zeros(1,length(conditionsToPlot))
        lengthOfTC=0
        aucArray=[]
        thisTCuppermost=[]
        for condition in conditionsToPlot
            labelSpeciesNames=[]
            lengthOfTC=size(DataFrame(CSV.File(folder*"/sol_"*condition*"_cell_1.csv")),2)-1
            conditionArray=zeros(last_cell,lengthOfTC)
            lineColor=colorArray[conditionIndex]
            virtExpFlag=false
            conditionTC=[]
            for i in first_cell:last_cell
                thisCellData=DataFrame(CSV.File(folder*"/sol_"*condition*"_cell_"*string(i)*".csv"))
                if !("names" in names(thisCellData))
                    insertcols!(thisCellData, 1, :names=>syms)
                end
                allNoneFloats=findall(eltype.(eachcol(thisCellData)).!=Float64)
                if length(allNoneFloats)>1
                    for index in allNoneFloats[2:end]
                        thisCellData[!,index]=parse.(Float64,thisCellData[:,index])
                    end
                end
                datapointsinTC=0
                thisTC=zeros(1,size(thisCellData,2)-1)
                if endswith(species,"*")
                    println("* star detected")
                    virtExpFlag=true
                    speciesShort=species[1:end-1]
                    speciesIDs=intersect(findall( x ->occursin(speciesShort,x),syms),findall(x->!startswith(x,"t"),syms))
                    speciesNames=String.(syms[speciesIDs])
                    labelSpeciesNames=speciesNames
                    datapointsinTC=size(speciesNames,1)
                    thisTC=zeros(size(speciesNames,1),size(thisCellData,2)-1)
                    println("For species: "*species*" printing: ")
                    n=1
                    for thisName in speciesNames
                        thisSpeciesTC=Matrix(thisCellData[thisCellData[!,:names].==thisName,:])[2:end]
                        thisTC[n,:]=thisSpeciesTC
                        n=n+1
                    end
#                     return(thisTC)
                    thisTCuppermost=thisTC

                else
                    println(speciesToPlot, "will be given a separate bar. Function is intended for a species ending in asterisk")
                    thisTC=Matrix(thisCellData[thisCellData[!,:names].==species,:])[2:end]
                end
                conditionArray[i,:]=thisTC[1:lengthOfTC]
            end
            df = DataFrame(Float64.(conditionArray),:auto)
            meanOfCondition=mean(conditionArray, dims=1)
            stdOfCondition=std(conditionArray, dims=1)
            plot!(thisPlotStd,meanOfCondition',grid=false,color=lineColor,ribbon=stdOfCondition',fillalpha=.25,label=condition,linewidth=5)
            meansOfAllConditions[conditionIndex]=meanOfCondition[1]
            stdOfAllConditions[conditionIndex]=stdOfCondition[1]
            conditionIndex+=1
            auc = sum(meanOfCondition)/60
            push!(aucArray,auc)
            aucOfTC=Array{Float64, 2}(undef, size(thisTCuppermost,1),1)
            for row in 1:size(thisTCuppermost,1)
                for timepoint in thisTCuppermost[row,:]
                    aucOfTC[row]=sum(thisTCuppermost[row,:])
                end
            end
            zlist=[]
            z=reshape(aucOfTC,(1,size(aucOfTC,1)))
            push!(zlist,z)
            labels=permutedims(labelSpeciesNames[:,:])
#             println(labels)
            stackedbar=groupedbar(z,bar_position=:stack,label=labels,bar_width=1,title=condition,size=(500,400),legend=:outerbottomright)
            display(stackedbar)

        end
        
#         conditionIndex=1
#         for condition in conditionsToPlot
# #             for n in size(thisTC,1)
                
# #             end
#             bar!(boxPlotAllTC,[conditionIndex],  )
#             bar!(boxPlotAll,[conditionIndex], [meansOfAllConditions[conditionIndex]], c=colorArray[conditionIndex], yerr = stdOfAllConditions[conditionIndex], label = false,xrotation = 70,fillcolor=:match)
#             conditionIndex+=1
# #             groupedbar!(stackedbar,z,bar_position=:stack,label=testspeciesNames,xticks=(1:size(conditions,1),conditions[conditionIndex]))
#         end
#         zlist=[]
#         z=reshape(aucOfTC,(1,size(aucOfTC,1)))
#         push!(zlist,z)
#         stackedbar=groupedbar(z,bar_position=:stack)#,label=testspeciesNames,bar_width=0.2)
#         display(stackedbar)
#         plot!(boxPlotAll,xticks = (1:length(conditionsToPlot), conditionsToPlot),xlim=(0,length(conditionsToPlot)+1),ylim=(0,maxValOfYAxis),dpi=300,size=(round(800*qualityScaling),round(1000*qualityScaling)),xtickfontsize=15,ytickfontsize=18,titlefontsize=18)        
#         display(boxPlotAll,boxPlotAllTC)
    end
end

stackedBar (generic function with 1 method)

In [25]:
stackedBar(speciesToPlot,["noncanon_ikkSS1_nikTC1","noncanon_ikkSS1000_nikTC1000"],colorArray,first_cell,1,folder,hoursToPlot,maxValOfYAxis,qualityScaling)

* star detected
For species: RelA* printing: 


* star detected
For species: RelA* printing: 


In [23]:
fiveCornerCanonConditions2=["canon_ikkTC1_NIKFuncSS05","canon_ikkTC1000_NIKFuncSS1000","canon_ikkTC10_NIKFuncSS10","canon_ikkTC1000_NIKFuncSS05","canon_ikkTC1_NIKFuncSS1000"]
fiveCornerNonCanonConditions2=["noncanon_ikkSS1_nikTC05","noncanon_ikkSS1000_nikTC1000","noncanon_ikkSS10_nikTC10","noncanon_ikkSS1000_nikTC05","noncanon_ikkSS1_nikTC1000"]
sevenCornerNonCanonConditions=["noncanon_ikkSS1_nikTC1","noncanon_ikkSS1000_nikTC1000","noncanon_ikkSS10_nikTC10","noncanon_ikkSS1000_nikTC1","noncanon_ikkSS1_nikTC1000","noncanon_ikkSS1_nikTC05","noncanon_ikkSS1000_nikTC05"]

7-element Vector{String}:
 "noncanon_ikkSS1_nikTC1"
 "noncanon_ikkSS1000_nikTC1000"
 "noncanon_ikkSS10_nikTC10"
 "noncanon_ikkSS1000_nikTC1"
 "noncanon_ikkSS1_nikTC1000"
 "noncanon_ikkSS1_nikTC05"
 "noncanon_ikkSS1000_nikTC05"

In [14]:
speciesToPlot=["RelA*"]
fiveCornerNonCanonConditions=["noncanon_ikkSS1_nikTC1","noncanon_ikkSS1000_nikTC1000","noncanon_ikkSS10_nikTC10","noncanon_ikkSS1000_nikTC1","noncanon_ikkSS1_nikTC1000"]
colorArray=palette(:seaborn_colorblind)
first_cell,last_cell=1,1
hoursToPlot=9
maxValOfYAxis=25
qualityScaling=1
stackedBar(speciesToPlot,sevenCornerNonCanonConditions,colorArray,first_cell,1,folder,hoursToPlot,maxValOfYAxis,qualityScaling)

* star detected
For species: RelA* printing: 


* star detected
For species: RelA* printing: 


* star detected
For species: RelA* printing: 


* star detected
For species: RelA* printing: 


* star detected
For species: RelA* printing: 


* star detected
For species: RelA* printing: 


* star detected
For species: RelA* printing: 


In [19]:
speciesToPlot=["RelA*" "RelB*"]
# speciesToPlot=[""]
fiveCornerCanonConditions=["canon_ikkTC1_NIKFuncSS1","canon_ikkTC1000_NIKFuncSS1000","canon_ikkTC10_NIKFuncSS10","canon_ikkTC1000_NIKFuncSS1","canon_ikkTC1_NIKFuncSS1000"]
stackedBar(speciesToPlot,fiveCornerCanonConditions2,colorArray,first_cell,1,folder,hoursToPlot,maxValOfYAxis,qualityScaling)

LoadError: MethodError: no method matching *(::Vector{String}, ::Vector{String})
[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:591
[0m  *(::StridedVecOrMat, [91m::LinearAlgebra.Adjoint{<:Any, <:LinearAlgebra.LQPackedQ}[39m) at ~/.local/julia-1.8.3/share/julia/stdlib/v1.8/LinearAlgebra/src/lq.jl:269
[0m  *(::StridedVecOrMat, [91m::LinearAlgebra.LQPackedQ[39m) at ~/.local/julia-1.8.3/share/julia/stdlib/v1.8/LinearAlgebra/src/lq.jl:293
[0m  ...

In [21]:
speciesToPlot=["RelA*"]
# speciesToPlot=[""]
fiveCornerCanonConditions=["canon_ikkTC1_NIKFuncSS1","canon_ikkTC1000_NIKFuncSS1000","canon_ikkTC10_NIKFuncSS10","canon_ikkTC1000_NIKFuncSS1","canon_ikkTC1_NIKFuncSS1000"]
stackedBar(speciesToPlot,sevenCornerNonCanonConditions,colorArray,first_cell,1,folder,hoursToPlot,maxValOfYAxis,qualityScaling)

* star detected
For species: RelA* printing: 


* star detected
For species: RelA* printing: 


* star detected
For species: RelA* printing: 


* star detected
For species: RelA* printing: 


* star detected
For species: RelA* printing: 


* star detected
For species: RelA* printing: 


* star detected
For species: RelA* printing: 


Things wrong with this:  

It needs some kind of labels.

It needs to not be dependent on the testTC bit. (obtained by running with the returnTC bit.)

I could separate out my bits so that I take the code UP TO return TC.

Then stick the rest of the code AFTER

In [6]:
aucOfTC=Array{Float64, 2}(undef, size(testTC,1),1)
for row in 1:size(testTC,1)
    for timepoint in testTC[row,:]
        aucOfTC[row]=sum(testTC[row,:])
    end
end


LoadError: UndefVarError: testTC not defined